In [60]:
import requests as reqs
import json
import pandas as pd

In [206]:
year = 2017
region = "VelikiiNovgorod"

In [216]:
data = {
    "date": [f"MONTHS:{m}.{year}" for m in range(1, 13)],
    "ParReg": '49',
    'order': {
        "type" :1, "fieldName": 'dat'
    },
    "reg": '49401',
    'ind': '1',
    'st': '1',
    'en': '405'
}

In [217]:
card_json = {"data": data}
card_json['data'] = json.dumps(data, separators=(',', ':')).encode('utf8').decode("unicode-escape")

In [218]:
response = reqs.post("http://stat.gibdd.ru/map/getDTPCardData", json=card_json)

In [219]:
response

<Response [200]>

In [214]:
response_vnovgorod = response.json()['data']

In [201]:
json_nov = json.loads(response_vnovgorod)

In [202]:
region = json_nov['RegName']
cards = json_nov['tab']

In [203]:
PROCESSED_CARDS = []
for card in cards:
    processed_card = {}
    info = card.pop("infoDtp") if "infoDtp" in card.keys() else {}
    ts_info = info.pop("ts_info") if info else {}
    uchInfo = info.pop("uchInfo") if info else []
    processed_card.update(**card, **info, )
    PROCESSED_CARDS.append(processed_card)

In [204]:
df = pd.DataFrame.from_dict(PROCESSED_CARDS)
df = df.rename(columns={
    "KartId": "id",
    "rowNum": "row",
    "DTP_V": "crash_type",
    "POG": "deceased",
    "RAN": "wounded",
    "K_TS": "vehicles",
    "K_UCH": "participants",
    "ndu": "road_deficiency",
    "s_pog": "weather",
    "osv": "light_conditions",
    "s_pch": "road_conditions",
    "n_p": "settlement",
    "dor": "main_road",
    "dor_k": "road_category",
    "dor_z": "road_significance",
    "k_ul": "street_category",
    "sdor": "onsite_crash_road_objects",
    "COORD_W": "latitude", 
    "COORD_L": "longitude",
})

In [205]:
df.to_excel(f"{region}{year}.xlsx")